# Vision

This notebook creates training images for CV and RL models, the socket listener will listen an indicator that a song has started in the game and records the gameplay footage at n FPS then split the frames into individual images to the output folder

In [42]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
import mss
import time
import os
import warnings
import logging
import json
from concurrent.futures import ThreadPoolExecutor
import pathlib
from helper import detect, capture, split_frames, SocketListener
import traceback

In [43]:
# CONSTANTS

VID_OUTPUT_DIR = 'video_out'        # Directory to save video output
SAVE_OUTPUT = False                 # Whether or not to save the video output
FPS = 15                            # Frames to capture per second

In [44]:
listener = SocketListener()
listener.start()

Listening on 127.0.0.1:5555


In [45]:
frames_to_capture = FPS * 3
frames_captured = 0
intro = True

monitor_1 = mss.mss().monitors[2]
print(f'Monitor: {monitor_1}')
t, l, w, h = monitor_1['top'], monitor_1['left'], monitor_1['width'], monitor_1['height']
region_1 = {'left': l+int(w * 0.338), 'top': t, 'width': w-int(w * 0.673), 'height': h} # Gameplay region
# region_2 = {'left': l+int(w * 0.67), 'top': t, 'width': w-int(w * 0.98), 'height': h-int(h * 0.33)} # Judgement counter region
# region_3 = {'left': l+int(l * 0.859), 'top': t, 'width': w-int(w * 0.820), 'height': h-int(h * 0.952)} # Score region

print(f'Capturing {region_1}')

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_1 = cv2.VideoWriter(f'{VID_OUTPUT_DIR}/output_1.mp4', fourcc, FPS, (region_1['width'], region_1['height']))

with ThreadPoolExecutor(max_workers=8) as executor:
  while listener.is_listening or listener.is_first_connection:  
    if listener.has_connection:
      if intro:
        print('Waiting for intro to finish')
        time.sleep(3)
        intro = False
        
      print(f'Captured frame {frames_captured}')
      time_start = time.time()
      thread_1 = executor.submit(capture, region_1)
      img_1 = thread_1.result()
      
      out_1.write(cv2.cvtColor(np.array(img_1), cv2.COLOR_BGRA2BGR))
      frames_captured += 1
      # If the current loop finishes faster than the FPS, wait.
      if time_start + 1/FPS > time.time():
        time.sleep(time_start + 1/FPS - time.time())
    elif not listener.is_first_connection and not listener.has_connection:
      print('Song ended')
      listener.stop()
      if listener.is_listening:
        print('Force stop')
        listener._stop()
      break

out_1.release()
print(f'Capture region saved to {VID_OUTPUT_DIR}/output_1.mp4')

Monitor: {'left': 0, 'top': 0, 'width': 1920, 'height': 1080}
Capturing {'left': 648, 'top': 0, 'width': 628, 'height': 1080}
Waiting for intro to finish
Captured frame 0
Captured frame 1
Captured frame 2
Captured frame 3
Captured frame 4
Captured frame 5
Captured frame 6
Captured frame 7
Captured frame 8
Captured frame 9
Captured frame 10
Captured frame 11
Captured frame 12
Captured frame 13
Captured frame 14
Captured frame 15
Captured frame 16
Captured frame 17
Captured frame 18
Captured frame 19
Captured frame 20
Captured frame 21
Captured frame 22
Captured frame 23
Captured frame 24
Captured frame 25
Captured frame 26
Captured frame 27
Captured frame 28
Captured frame 29
Captured frame 30
Captured frame 31
Captured frame 32
Captured frame 33
Captured frame 34
Captured frame 35
Captured frame 36
Captured frame 37
Captured frame 38
Captured frame 39
Captured frame 40
Captured frame 41
Captured frame 42
Captured frame 43
Captured frame 44
Captured frame 45
Captured frame 46
Captured f

Traceback (most recent call last):
  File "c:\Users\tiany\Desktop\Work\Dev\UB CLASS\FALL 2024\RL\final project\helper.py", line 123, in _listen
    conn, addr = self.sock.accept()
  File "c:\Users\tiany\AppData\Local\Programs\Python\Python310\lib\socket.py", line 293, in accept
    fd, addr = self._accept()
OSError: [WinError 10038] An operation was attempted on something that is not a socket


In [ ]:
WRITE_META_DATA = True
video_path = './video_out/output_1.mp4'
frame_out_main_path = 'frames/rl_simulated_training_sample'
sub_path_idx = int(time.time())
meta_data = {
  'song_name': 'Where Our Blue Is (TV size)',
  'song_duration': 86,
  'note_count': 353,
  'hold_note_count': 123,
  'difficulty': 2.02,
}

while os.path.exists(f'{frame_out_main_path}/{sub_path_idx}'):
    sub_path_idx += 1

complete_frame_out_path = f'{frame_out_main_path}/{sub_path_idx}'
os.makedirs(complete_frame_out_path)

try:
  split_frames(video_path, complete_frame_out_path)
except Exception as e:
  print(traceback.format_exc())
  os.rmdir(complete_frame_out_path)
  
if WRITE_META_DATA:
  with open(f'{complete_frame_out_path}/metadata.json', 'w') as f:
    json.dump(meta_data, f)

Wrote 1525 frames to /frames/rl_simulated_training_sample/1732780928
